In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from nnfs import moons

In [ ]:
X,Y = moons()
sns.scatterplot(X[:,0], X[:,1], hue=Y)

In [ ]:
def sigmoid(a):
    return 1.0 / (1.0 + np.exp(-a))

In [ ]:
def plotfn(f, lo=-5, hi=5):
    pts = np.linspace(lo, hi, num=100)
    sns.lineplot(x=pts, y=list(map(f, pts)))

In [ ]:
plotfn(sigmoid)

In [ ]:
class Layer(object):
    def loss(self):
        return 0
    
class FC(Layer):
    "Fully connected layer"
    def __init__(self, d_in, d_out):
        self.d_in = d_in
        self.d_out = d_out
        self.W = np.random.randn(d_in, d_out) / 100
        self.b = np.zeros(d_out)
        
    def forward(self, x):
        # x @ W + b
        self.dW = x.T
        return (x @ self.W) + self.b
    
    def backward(self, grad_in):
        dW = self.dW @ grad_in
        dW += 0.001 * self.W
        db = grad_in.sum(axis=0)
        db += 0.001 * self.b
        self.W -= 0.1 * dW
        self.b -= 0.1 * db
        return grad_in @ self.W.T
    
    def loss(self):
        return 0.005 * (self.W * self.W).sum()
    
class Relu(Layer):
    def forward(self, x):
        self.grads = x >= 0
        return x * self.grads
    def backward(self, grad_in):
        return grad_in * self.grads
    
class Softmax(Layer):
    def forward(self, x):
        x = x - np.max(x, axis=-1, keepdims=True)
        exps = np.exp(x)
        probs = exps / exps.sum(axis=-1, keepdims=True)
        self.probs = probs
        return probs
    def backward(self, grad_in):
        return grad_in

class NN:
    def __init__(self, *layers):
        self.layers = layers
    
    def train(self, X, Y):
        n_examples = len(X)
        
        v = X
        for layer in self.layers:
            v = layer.forward(v)
        
        probs = v
        cel = -np.log(probs[np.arange(n_examples), Y])
        L = cel.sum() / n_examples
        
        z = np.zeros_like(probs)
        z[np.arange(n_examples), Y] += 1
        grad = (probs - z) / n_examples
        for layer in reversed(self.layers):
            grad = layer.backward(grad)

        return L
        
    def predict(self, x):
        v = x
        for layer in self.layers:
            v = layer.forward(v)
        return np.argmax(v, axis=-1)

In [ ]:
nn = NN(
    FC(2,20),
    Relu(),
    FC(20,20),
    Relu(),
    FC(20,2),
    Softmax())

In [ ]:
X,Y = ds.make_moons(n_samples=100, shuffle=True, noise=0.2)
nn.predict(X)

In [ ]:
losses = []

In [ ]:
for i in range(300):
    for _ in range(100):
        losses.append(nn.train(X,Y))

In [ ]:
plt.plot(losses)

In [ ]:
xrange=np.linspace(-1.5, 2.5, 100)
yrange=np.linspace(-1.5, 1.5, 100)
pts = np.array([[x,y] for x in xrange for y in yrange])
print(pts.shape)
C = nn.predict(pts)
print(C.shape)
plt.figure(figsize=(10,6))
sns.scatterplot(pts[:,0], pts[:,1], hue=C, palette='Accent')
sns.scatterplot(X[:,0], X[:,1], hue=Y)